In [1]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [2]:
data = tf.random.uniform((2,221,221,3),seed=1)
print(data)

Tensor("random_uniform:0", shape=(2, 221, 221, 3), dtype=float32)


In [3]:
W1 = tf.random.uniform((7,7,3,96))
b1 = tf.random.uniform((1,1,96))
W2 = tf.random.uniform((7,7,96,256))
b2 = tf.random.uniform((1,1,256))
W3 = tf.random.uniform((3,3,256,512))
b3 = tf.random.uniform((1,1,512))
W4 = tf.random.uniform((3,3,512,512))
b4 = tf.random.uniform((1,1,512))
W5 = tf.random.uniform((3,3,512,1024))
b5 = tf.random.uniform((1,1,1024))
W6 = tf.random.uniform((3,3,1024,1024))
b6 = tf.random.uniform((1,1,1024))
W7 = tf.random.uniform((5,5,1024,4096))
b7 = tf.random.uniform((1,1,4096))
W8 = tf.random.uniform((1,1,4096,4096))
b8 = tf.random.uniform((1,1,4096))
W9 = tf.random.uniform((1,1,4096,1000))
b9 = tf.random.uniform((1,1,1000))

In [4]:
#Conv1
conv1 = tf.nn.conv2d(data,W1,strides=(1,2,2,1),padding='VALID') + b1
#  relu....
pool1 = tf.nn.max_pool(conv1,ksize=(1,3,3,1),strides=(1,3,3,1),padding='VALID')

#Conv2
conv2 = tf.nn.conv2d(pool1,W2,strides=(1,1,1,1),padding="VALID")
pool2 = tf.nn.max_pool(conv2,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID")

#Conv3
conv3 = tf.nn.conv2d(pool2,W3,strides=(1,1,1,1),padding="SAME")

#Conv4
conv4 = tf.nn.conv2d(conv3,W4,strides=(1,1,1,1),padding="SAME")

#Conv5
conv5 = tf.nn.conv2d(conv4,W5,strides=(1,1,1,1),padding="SAME")


#Conv6
conv6 = tf.nn.conv2d(conv5,W6,strides=(1,1,1,1),padding="SAME")
print(conv6)
pool6 = tf.nn.max_pool(conv6,ksize=(1,3,3,1),strides=(1,3,3,1),padding="VALID")

#Connv7
conv7 = tf.nn.conv2d(pool6,W7,strides=(1,1,1,1),padding="VALID")

#conv8 
conv8 = tf.nn.conv2d(conv7,W8,strides=(1,1,1,1),padding="VALID")

#conv9
conv9 = tf.nn.conv2d(conv8,W9,strides=(1,1,1,1),padding="VALID")
conv9

Tensor("Conv2D_5:0", shape=(2, 15, 15, 1024), dtype=float32)


<tf.Tensor 'Conv2D_8:0' shape=(2, 1, 1, 1000) dtype=float32>

In [5]:
sess = tf.Session()

In [56]:
def offset(conv,W,sess):
    a = []
    for w in range(3):
        for h in range(3):
            print(h,w)
            conv_ = conv[:,h:,w:,:]
            pool_ = tf.nn.max_pool(conv,ksize=(1,3,3,1),strides=(1,3,3,1),padding="VALID")

            conv7 = tf.nn.conv2d(pool_,W7,strides=(1,1,1,1),padding="VALID")

            conv8 = tf.nn.conv2d(conv7,W8,strides=(1,1,1,1),padding="VALID")
            out = tf.nn.conv2d(conv8,W9,strides=(1,1,1,1),padding="VALID")
            out = tf.squeeze(out)
            out_ = sess.run(out)
            a.append(out_)
    
    
    a = np.array(a)
    maps,h,w,c = a.shape
    a = a.reshape((3*h,3*w,c))
    
    return a

In [57]:
test1 = tf.random.uniform((1,281,317,3),seed=1)
#Conv1
conv1 = tf.nn.conv2d(test1,W1,strides=(1,2,2,1),padding='VALID') + b1
#  relu....
pool1 = tf.nn.max_pool(conv1,ksize=(1,3,3,1),strides=(1,3,3,1),padding='VALID')

#Conv2
conv2 = tf.nn.conv2d(pool1,W2,strides=(1,1,1,1),padding="VALID")
pool2 = tf.nn.max_pool(conv2,ksize=(1,2,2,1),strides=(1,2,2,1),padding="VALID")

#Conv3
conv3 = tf.nn.conv2d(pool2,W3,strides=(1,1,1,1),padding="SAME")

#Conv4
conv4 = tf.nn.conv2d(conv3,W4,strides=(1,1,1,1),padding="SAME")

#Conv5
conv5 = tf.nn.conv2d(conv4,W5,strides=(1,1,1,1),padding="SAME")
print(conv5)
out = offset(conv5,W7,sess)
print(out.shape)

Tensor("Conv2D_689:0", shape=(1, 20, 23, 1024), dtype=float32)
0 0
1 0
2 0
0 1
1 1
2 1
0 2
1 2
2 2
(6, 9, 1000)


In [62]:
from scipy import stats
def predict(out):
    out = out.reshape((-1,1000))
    argmax = np.argmax(out,axis=1)
    print(argmax)
    print(argmax.shape)
    res = stats.mode(argmax)[0][0]
    print(res)

In [63]:
predict(out)

[473 473 473 473 473 473 848 848 848 848 848 848 528 528 528 528 528 528
  66  66  66  66  66  66  50  50  50  50  50  50 798 798 798 798 798 798
 585 585 585 585 585 585 678 678 678 678 678 678 457 457 457 457 457 457]
(54,)
50


In [64]:
res = stats.mode([1,1,2,3,4])[0][0]
res

1